<a href="https://colab.research.google.com/github/yasaminesmati/seizuredetection-/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install

# section -one
ابتدا هر داده جدا خوانده شد.برای برطرف کردن نویز فیلتر  روی داده ها اعمال شد.سپس کانال های مورد نظر جداشدند.
برای هر داده دو دیتا ست ویژگی و خود سیگنال ایجاد شد.

In [ ]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.5 MB/s eta 0:00:00


#import


In [50]:
import keras
from keras.models import Sequential
from tensorflow.keras.backend import clear_session
from keras.layers import  Dense, Conv1D, Dropout, Flatten, BatchNormalization ,LeakyReLU,MaxPool1D,GlobalAveragePooling1D,AveragePooling1D, concatenate
from keras.callbacks import EarlyStopping

In [51]:
import numpy as np
import mne
import pandas as pd
import csv
import sys
import math
from scipy import stats
from scipy.fft import fft, ifft
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn import tree
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import keras
from keras.models import Sequential
from keras.layers import  Dense, Conv3D, Dropout, Flatten, BatchNormalization 
from keras.callbacks import EarlyStopping
from random import shuffle
import seaborn as sns
import re
import os
from pandas import read_csv
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# read the data

In [ ]:
edf_file_names="content/chb01_26.edf"
# base_path="/content/"
# subject_id=1
# summary_file = os.path.join(base_path, "chb{:02d}-summary.txt".format(subject_id, subject_id))
# summary_content = open(summary_file,'r').read()

In [ ]:
edf= mne.io.read_raw_edf('/content/chb03_01.edf',preload=True)
drop_ch= edf.ch_names[:16]
edf2=edf.drop_channels(drop_ch)
drop_ch_two=edf2.ch_names[2:]
edf2=edf2.drop_channels(drop_ch_two)
edf3=edf2.filter(l_freq=0.5,h_freq=45)
#epochs=mne.make_fixed_length_epochs(edf3,duration=10,overlap=1)
edf3=edf3.get_data()
len(edf3)

Extracting EDF parameters from /content/chb03_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


<ipython-input-361-396ec37b3688>:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf= mne.io.read_raw_edf('/content/chb03_01.edf',preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s finished


2

# feature extraction

In [ ]:
def mean_variance(df):
    variance_vals = np.var(df)
    return np.mean(variance_vals)

def median_variance(df):
  
    variance_vals = np.var(df)
    return np.median(variance_vals)

def max(df):
    return np.max(df)

def min(df):
    return np.min(df)

def ptp(df):
    return np.ptp(df)

def skew(x):
    return stats.skew(x)

def kurtosis(x):
    return stats.kurtosis(x)

def Mean(x):
  resp = np.mean(x)
  return resp

def Std(x):
  resp = np.std(x)
  return resp

def Power(x):
  F = np.fft.fft(x)
  P = F * np.conjugate(F)
  resp = sum(P)
  return resp

def fourier(x):
  resp = fft(x)
  return np.mean(resp)

In [ ]:
#window function to work over data and calculate features
def add_feature(edf_file, start, duration, seizure_start, seizure_end):
    d = edf_file[:,256*start:256*(start+duration)]
    row_to_add = []
    mean_var = mean_variance(d[0])
    
    med_var = median_variance(d[0])
    maximum = max(d[0])
    minimum = min(d[0])
    ptpp = ptp(d[0])
    skewness = skew(d[0])
    mean = Mean(d[0])
    std = Std(d[0])
    pow = Power(d[0])
    fft = fourier(d[0])
    
    if start in range(seizure_start, seizure_end):
        t = 1
    else:
        t = 0
    row_to_add.append(mean_var)
    row_to_add.append(med_var)
    row_to_add.append(maximum)
    row_to_add.append(minimum)
    row_to_add.append(ptpp)
    row_to_add.append(skewness)
    row_to_add.append(mean)
    row_to_add.append(std)
    row_to_add.append(pow)
    row_to_add.append(fft)

    row_to_add.append(start)
    row_to_add.append(t)
    return row_to_add

In [ ]:
#window function to work over data and calculate features
def add_signal(edf_file, start, duration, seizure_start, seizure_end):
    d = edf_file[:,256*start:256*(start+duration)]
    row_to_add = []
    
    if start in range(seizure_start, seizure_end):
        t = 1
    else:
        t = 0

    row_to_add.append(d[0])
    row_to_add.append(d[1])

    row_to_add.append(t)
    # print(row_to_add[2])
    return row_to_add

In [ ]:
df = pd.DataFrame()
signals = pd.DataFrame()

In [ ]:
from pandas.io import excel
#driver code
start = temp = 350
duration = 10
end = 420
seizure_start = 362      
seizure_end = 414     

# with open('Training_Data.csv', 'a') as file:
#     writer = csv.writer(file)
while temp < end:    
        row = add_feature(edf2, temp, duration, seizure_start, seizure_end)
        df = df.append(pd.Series(row), ignore_index=True)

        row2 = add_signal(edf3, temp, duration, seizure_start, seizure_end)
        signals = signals.append(pd.Series(row2), ignore_index=True)
        temp = temp +1


In [ ]:
signals.to_excel('./03-01-original.xlsx',columns=signals.columns)


In [ ]:
df.to_excel('./03-01-feature.xlsx',columns=df.columns)

#concat all the data

In [ ]:
one=pd.read_excel("/content/01-03-original.xlsx")
two=pd.read_excel("/content/01-04-original.xlsx")
three=pd.read_excel("/content/01-15-original.xlsx")
four=pd.read_excel("/content/01-16-original.xlsx")
five=pd.read_excel("/content/01-18-original.xlsx")
six=pd.read_excel("/content/01-21-original.xlsx")
seven=pd.read_excel("/content/01-26-original.xlsx")
eight=pd.read_excel("/content/02-16-original.xlsx")
nine=pd.read_excel("/content/02-16plus-original.xlsx")
ten=pd.read_excel("/content/02-19plus-original.xlsx")
eleven=pd.read_excel("/content/03-01-original.xlsx")

In [ ]:
final_original=pd.concat([one,two,three,four,five,six,seven,eight,nine,ten,eleven])

In [ ]:
final_original.to_excel('./final-original.xlsx',columns=final_original.columns)

In [ ]:
final_feature=pd.read_excel("/content/final-feature.xlsx")

In [ ]:
final_feature.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='object')

In [ ]:
final_feature.columns=['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','label']

In [ ]:
final_feature=final_feature.drop(columns=['f1','f2','f8','f11'])

In [ ]:
final_feature=final_feature.drop(columns=['f12'])

# feature extraction method

In [ ]:
final_original = pd.read_excel('final-original.xlsx')
final_feature_extraction = pd.read_excel('final_feature_extraction.xlsx')
final_feature = pd.read_excel('final-feature.xlsx')

In [ ]:
def correlation(dataset):
    corr_matrix = dataset.corr(method='pearson')
    return corr_matrix

In [ ]:
corr_features = correlation(final_feature)

In [ ]:
y=final_feature.loc[:, 11]
y

0       0
1       0
2       0
3       0
4       0
       ..
1555    0
1556    0
1557    0
1558    0
1559    0
Name: 11, Length: 1560, dtype: int64

In [ ]:
final_feature=final_feature.drop(columns=11)
final_feature

In [ ]:
accuracy_dict=dict()
for feature in final_feature:
    x_train,x_test,y_train,y_test=train_test_split(final_feature[feature].values.reshape(-1,1),y,random_state=42,test_size=0.2)
    classifier=tree.DecisionTreeClassifier().fit(x_train,y_train)
    prediction=classifier.predict(x_test)
    accuracy_test=accuracy_score(y_test,prediction)
    accuracy_dict[feature]=accuracy_test   

ValueError: ignored

In [ ]:
accuracy_dict

{'f3': 0.5416666666666666,
 'f4': 0.5416666666666666,
 'f5': 0.9198717948717948,
 'f6': 0.9102564102564102,
 'f7': 0.8878205128205128,
 'f9': 0.5544871794871795,
 'f10': 0.6923076923076923,
 'f13': 0.8141025641025641}

In [ ]:
def select_feature(chosen_features: list):
    chosen = (None, 0)
    candidate = (None, 0)
    for feature in final_feature:
        if feature not in chosen_features:
            corelationtest = 1
            for features in chosen_features:
                if corr_features[feature][features] < corelationtest:
                    corelationtest = corr_features[feature][features]
            corelationtest = 1 - corelationtest
            f1 = 2*((corelationtest* accuracy_dict[feature])/(corelationtest+accuracy_dict[feature]))
            # return(f1)
            candidate = (feature, f1)
        if candidate[1] > chosen[1]:
            chosen = candidate
            print(chosen)
    return chosen[0]


In [ ]:
chosen_features = ['f6']
for i in range(6):
    chosen_features.append(select_feature(chosen_features))
print(chosen_features)

('f3', 0.8290728319072462)
('f5', 1.2212107223142277)
('f3', 0.8290728319072462)
('f7', 1.2202144635014978)
('f3', 0.8290728319072462)
('f10', 0.9971517310328817)
('f3', 0.8290728319072462)
('f13', 0.9483067097223454)
('f3', 0.8290728319072462)
('f4', 0.8290728319072462)
['f6', 'f5', 'f7', 'f10', 'f13', 'f3', 'f4']


In [ ]:
final_feature.columns


Index(['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11',
       'f12', 'f13', 'label'],
      dtype='object')

In [ ]:
final_feature_extraction=final_feature.drop(columns=['f1','f2','f8','f9','f11','f12'])

In [ ]:
final_feature_extraction.columns

Index(['f3', 'f4', 'f5', 'f6', 'f7', 'f10', 'f13', 'label'], dtype='object')

In [ ]:
final_feature_extraction.to_excel('./final_feature_extraction.xlsx',columns=final_feature_extraction.columns)

# cnn


In [65]:
final_original.shape

(1560, 3)

In [76]:
def cnnmodel():
  clear_session()
  model=Sequential()

  x1 = Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(1560, 3))(final_original)
  x2 = Conv1D(filters=5,kernel_size=3,strides=1)(final_feature_extraction)
  x = concatenate([x1,x2])
  
  model.add(x1)
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(MaxPool1D(pool_size=2,strides=2))

  model.add(x1)
  model.add(LeakyReLU())
  model.add(MaxPool1D(pool_size=2,strides=2))
  model.add(Dropout(0.5))

  model.add(x)
  model.add(LeakyReLU())
  model.add(AveragePooling1D(pool_size=2,strides=2))
  model.add(Dropout(0.5))


#   model.add(Conv1D(filters=5,kernel_size=3,strides=1))
#   model.add(LeakyReLU())
#   model.add(AveragePooling1D(pool_size=2,strides=1))

#   model.add(Conv1D(filters=5,kernel_size=3,strides=1))
#   model.add(LeakyReLU())
#   model.add(GlobalAveragePooling1D())

  model.add(Dense(256, activation='sigmoid'))
  model.add(Dropout(0.5))
  model.add(Dense(2,activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


In [ ]:
model=cnnmodel()
model.summary()

In [ ]:
final_original=pd.read_excel("/content/final-original.xlsx")
final_feature_extraction=pd.read_excel("/content/final_feature_extraction.xlsx")

In [ ]:
print(final_original.shape)

(1560, 4)


In [ ]:
final_feature_extraction.drop(columns = ['Unnamed: 0'], inplace = True)
final_original.drop(columns = ['Unnamed: 0'], inplace = True)
final_original.columns

Index([0, 1, 2], dtype='object')

In [ ]:
final_original.columns=['f1','f2','label']
# final_original=final_original.drop(columns=['f1','f2'])

In [ ]:
X=final_original.drop(columns='label')
y=final_original['label']

In [ ]:
final_feature_extraction.columns

Index(['f3', 'f4', 'f5', 'f6', 'f7', 'f10', 'f13', 'label'], dtype='object')

In [ ]:
final_feature_extraction.columns=['f3','f4','f5','f6','f7','f10','13','label']
#final_feature_extraction=final_feature_extraction.drop(columns=['f1','f2'])

In [ ]:
final_feature_extraction

,f3,f4,f5,f6,f7,f10,13,label
0,1.171130e-09,1.171130e-09,0.000098,-0.000200,0.000298,0.000034,2900,0
1,1.130703e-09,1.130703e-09,0.000098,-0.000200,0.000298,0.000034,2901,0
2,1.095033e-09,1.095033e-09,0.000098,-0.000200,0.000298,0.000033,2902,0
3,1.137828e-09,1.137828e-09,0.000098,-0.000200,0.000298,0.000034,2903,0
4,8.788463e-10,8.788463e-10,0.000098,-0.000108,0.000206,0.000030,2904,0
...,...,...,...,...,...,...,...,...
1555,5.260746e-09,5.260746e-09,0.000310,-0.000197,0.000507,0.000073,415,0
1556,5.006398e-09,5.006398e-09,0.000310,-0.000158,0.000467,0.000071,416,0
1557,4.878292e-09,4.878292e-09,0.000310,-0.000158,0.000467,0.000070,417,0
1558,4.954571e-09,4.954571e-09,0.000310,-0.000166,0.000476,0.000070,418,0


In [ ]:
model.fit